In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import os

In [2]:
connect_info = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format('root','benbilly3@','localhost',3306,'settleProcess')  #1
engine = create_engine(connect_info)

# 確認清單

In [3]:
def afterSettleList(session,method='append'):
    db = pymysql.connect('localhost', 'root', 'benbilly3@','settleProcess', charset='utf8')
    cursor=db.cursor()
    sql="DELETE FROM AFTERSETTLELIST WHERE SESSION="+str(session)
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()
        
    df=pd.read_sql('select * from confirmPurchaseRight where session='+str(session),con=engine,index_col=['session','fbuid'])

    dfx=df.loc[:,['username','getSure','wineID','wineName','limit','orginalOrderCount','orderCount','giftBottle','orderCountAfterSale',
                 'price','totalPrice']]
    dfx['limit']=dfx['limit'].apply(lambda s:"限加"+str(s) if s>0 else '無限加限制')
    df_sorry=dfx[dfx['getSure']=='sorry']
    df_sorry['orderCountAfterSale']=0
    df_sorry['totalPrice']=0

    df_get=dfx[dfx['getSure']=='get']

    final_list=pd.concat([df_get,df_sorry])
    final_list=final_list.rename(columns={'username':'FB姓名','getSure':'購買權確認','wineID':'商品編號','wineName':'商品名稱',
                          'limit':'購買限制','orginalOrderCount':'留言購買數','orderCount':'結算確認購買數量','giftBottle':'優惠贈送量',
                         'orderCountAfterSale':'扣除贈送量後計價數','price':'商品單價','totalPrice':'商品總價'})
    final_list['備註']=['未排到' if (a=='sorry') 
                      else b if (a=='get')&(b!='無限加限制')&(c>0)
                      else '留言購買數大於庫存數,修正為最大可買數。' if (a=='get')&(b=='無限加限制')&(c>d) 
                      else '已確認'
                      for a,b,c,d in zip(final_list['購買權確認'],final_list['購買限制'],final_list['留言購買數'],final_list['結算確認購買數量'])]
    final_list=final_list.reset_index()
    final_list.to_sql(name='afterSettleList', con=engine, if_exists=method)

    return final_list.sort_index()

afterSettleList('20190812')


/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,session,fbuid,FB姓名,購買權確認,商品編號,商品名稱,購買限制,留言購買數,結算確認購買數量,優惠贈送量,扣除贈送量後計價數,商品單價,商品總價,備註
0,20190812,578924930,Gary Chan,get,4473,Tenuta San Guido 2015 le Difese,無限加限制,1,1,0,1,680,680,已確認
1,20190812,100000646546186,Derek Lee,get,4473,Tenuta San Guido 2015 le Difese,無限加限制,3,3,0,3,680,2040,已確認
2,20190812,1829989153,蔡明儒,get,4473,Tenuta San Guido 2015 le Difese,無限加限制,2,2,0,2,680,1360,已確認
3,20190812,100000034591181,Chia-chen Szu,get,4473,Tenuta San Guido 2015 le Difese,無限加限制,1,1,0,1,680,680,已確認
4,20190812,100002328014225,Wei Little,get,4473,Tenuta San Guido 2015 le Difese,無限加限制,2,2,0,2,680,1360,已確認
5,20190812,1152548412,Michael Wang,get,4473,Tenuta San Guido 2015 le Difese,無限加限制,2,2,0,2,680,1360,已確認
6,20190812,1452884049,Chung Hsien Hsieh,get,4473,Tenuta San Guido 2015 le Difese,無限加限制,2,2,0,2,680,1360,已確認
7,20190812,1487117625,Benjamin Chang,get,4473,Tenuta San Guido 2015 le Difese,無限加限制,2,2,0,2,680,1360,已確認
8,20190812,100001478964518,Rex Chen,get,4473,Tenuta San Guido 2015 le Difese,無限加限制,6,6,0,6,680,4080,已確認
9,20190812,100000251487356,Chih-sheng Huang,get,4473,Tenuta San Guido 2015 le Difese,無限加限制,3,3,0,3,680,2040,已確認


# 結算金額資訊

In [4]:
def feeList(session,method='append'):
    
    db = pymysql.connect('localhost', 'root', 'benbilly3@','settleProcess', charset='utf8')
    cursor=db.cursor()
    sql="DELETE FROM FEELIST WHERE SESSION="+str(session)
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()
        
    df=pd.read_sql('select * from calDiliverCostFinal where session='+str(session),con=engine,index_col=['session','fbuid'])
    #製作訂單編號
    df['index']=(df['index']+1).apply(lambda s:'000'+str(s) if s<10 else '00'+str(s) if s<100  else '0'+str(s) if s<1000 else str(s))
    df['訂單編號']=str(session)+df['index']
    df=df.loc[:,['訂單編號','orderCount','deilverFreeAll','deilverFreeAllbottle','discountAll','selfGetCost','diliverCostAfterDiscountAll',
                'diliverCostAfterDeilverFreeAll','totalPriceIncludeDC']]

    df['deilverFreeAll']=df['deilverFreeAll'].apply(lambda s:"滿"+str(s)+'元免運' if s>0 else '無')
    df['deilverFreeAllbottle']=df['deilverFreeAllbottle'].apply(lambda s:"滿"+str(s)+'件免運' if s>0 else '無')
    df['discountAll']=df['discountAll'].apply(lambda s:"全團"+str(s*10)+'折優惠' if s!=1 else '無全團折扣優惠')

    df=df.rename(columns={'orderCount':'結算確認購買數量','deilverFreeAll':'金額達標免運優惠','deilverFreeAllbottle':'瓶數達標免運優惠',
                          'discountAll':'全團折扣優惠','selfGetCost':'折扣前價格','diliverCostAfterDiscountAll':'折扣後自取價',
                          'diliverCostAfterDeilverFreeAll':'運費','totalPriceIncludeDC':'含運費總價'})
    df=df.reset_index()
    df.to_sql(name='feeList', con=engine, if_exists=method)

    return df.sort_index()

feeList('20190812')


,session,fbuid,訂單編號,結算確認購買數量,金額達標免運優惠,瓶數達標免運優惠,全團折扣優惠,折扣前價格,折扣後自取價,運費,含運費總價
0,20190812,506944678,201908120001,6,無,滿6件免運,無全團折扣優惠,4100,4100,0,4100
1,20190812,517368306,201908120002,3,無,滿6件免運,無全團折扣優惠,2620,2620,225,2845
2,20190812,526487238,201908120003,1,無,滿6件免運,無全團折扣優惠,2990,2990,160,3150
3,20190812,560410780,201908120004,8,無,滿6件免運,無全團折扣優惠,12360,12360,0,12360
4,20190812,578924930,201908120005,5,無,滿6件免運,無全團折扣優惠,3310,3310,225,3535
5,20190812,581492472,201908120006,1,無,滿6件免運,無全團折扣優惠,1580,1580,160,1740
6,20190812,591981161,201908120007,4,無,滿6件免運,無全團折扣優惠,2740,2740,225,2965
7,20190812,592053470,201908120008,6,無,滿6件免運,無全團折扣優惠,5990,5990,0,5990
8,20190812,595038932,201908120009,8,無,滿6件免運,無全團折扣優惠,5850,5850,0,5850
9,20190812,620610686,201908120010,5,無,滿6件免運,無全團折扣優惠,3200,3200,225,3425


# 會員更新

In [34]:
#抓會員資料
session='20190812'
df=pd.read_sql('select * from memberHelper ',con=engine)
df2=pd.read_sql('select * from afterSettleList where session='+str(session),con=engine,index_col=['fbuid'])
df3=pd.read_sql('select * from feeList where session='+str(session),con=engine,index_col=['fbuid'])
df=df.set_index(['FbId'])
helper_select=df.reindex(df3.index)
helper_select=helper_select[helper_select['helper'].isna()]

# 分配小幫手
n1=round(len(helper_select)*0.6)
n2=round(len(helper_select)*0.3)
helper_select.iloc[:n1,-1]='佩玄'
helper_select.iloc[n1:n1+n2,-1]='陳曉丰'
helper_select.iloc[n1+n2:,-1]='徐筱熙'

a=df2['FB姓名'].sort_values()
helper_select['Name']=a[~a.duplicated()]
helper_new=pd.concat([df,helper_select])
helper_new=helper_new.reset_index()
helper_new=helper_new.rename(columns={'level_0':'FbId'})
helper_new=helper_new.loc[:,['FbId','Name','userprofile','email','helper']]
helper_new.to_sql(name='memberHelper', con=engine, if_exists='replace')
# df.to_csv('/Users/benbilly3/Downloads/所有會員_小幫手(更新).csv')

,FbId,Name,userprofile,email,helper
0,1.000018e+14,鍾大衛,https://www.facebook.com/davidchung66,Davidi@gpiled.com,陳曉丰
1,1.665944e+09,Albert Hu,https://www.facebook.com/alberthhtsung,None,陳曉丰
2,8.379572e+08,WU Wayne,https://www.facebook.com/gcboytw,gcboy.tw@gmail.com,佩玄
3,1.000025e+14,潘宥瑞,https://www.facebook.com/profile.php?id=100002...,panyuray52@yahoo.com.tw,佩玄
4,1.544277e+09,Barkiel Huang,https://www.facebook.com/barkiel.huang,hallo33@gmail.com,佩玄
5,7.622746e+08,Honda Chou,https://www.facebook.com/honda.chou.10,hondachou@hotmail.com,佩玄
6,1.000003e+14,Andrew Chen,https://www.facebook.com/andrew.chen.92,catwing501@gmail.com,佩玄
7,6.875305e+08,Jaefo Liang,https://www.facebook.com/jaefo,jaefoliang@gmail.com,佩玄
8,1.086581e+09,鄭宇宏,https://www.facebook.com/YuHungCheng,m106537@me.com,佩玄
9,1.000056e+14,Edward Shih,https://www.facebook.com/profile.php?id=100005...,None,陳曉丰


#  結算後大表單輸出

In [36]:
def afterSettleBigTable(session):
    db = pymysql.connect('localhost', 'root', 'benbilly3@','settleProcess', charset='utf8')
    cursor=db.cursor()
    sql="DELETE FROM AFTERSETTLEBIGTABLE WHERE SESSION="+str(session)
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()
        
    #讀取資料
    df=pd.read_sql('select * from feeList where session='+str(session),con=engine,index_col=['fbuid'])
    df2=pd.read_sql('select * from afterSettleList where session='+str(session),con=engine,index_col=['fbuid'])
    helper=pd.read_sql('select * from memberHelper',con=engine,index_col=['FbId'])

    df2=df2[df2['商品總價']!=0]
    df2['訂單編號']=df['訂單編號']
    df2=df2.loc[:,['session','訂單編號','FB姓名','商品編號','商品名稱','商品單價','留言購買數','結算確認購買數量','商品總價','備註']]

    #結算金額對齊購買酒款清單欄位
    dfReset=df.loc[:,['session','訂單編號','結算確認購買數量','折扣後自取價','運費','含運費總價']]
    dfReset['FB姓名']='總計款數'
    dfReset['商品名稱']='自取付款金額'
    dfReset['留言購買數']='運費'
    dfReset['商品總價']='宅配付款金額'
    dfReset=dfReset.rename(columns={'結算確認購買數量':'商品編號','折扣後自取價':'商品單價','運費':'結算確認購買數量','含運費總價':'備註'})
    dfReset=dfReset.loc[:,df2.columns]

    #合併資料,加入小幫手資訊
    fs=pd.concat([df2,dfReset])
    fs['小幫手']=helper['helper']
    fs=fs.reset_index()

    fs=fs.set_index('小幫手')
    fs=fs.sort_values(['fbuid','訂單編號'])
    fs=fs.reset_index()
    fs['訂單編號']=fs['訂單編號'].astype(str)
    fs.to_sql(name='afterSettleBigTable', con=engine, if_exists='replace')
    return fs

df=afterSettleBigTable('20190812')
# df=df[df['小幫手'].isna()]
# df.to_csv('/Users/benbilly3/Downloads/20190812佩玄快閃團結算後大表單.csv')
df

,小幫手,fbuid,session,訂單編號,FB姓名,商品編號,商品名稱,商品單價,留言購買數,結算確認購買數量,商品總價,備註
0,徐筱熙,506944678,20190812,201908120001,Derek Chen,4473,Tenuta San Guido 2015 le Difese,680,2,2,1360,已確認
1,徐筱熙,506944678,20190812,201908120001,Derek Chen,4507,Joseph Drouhin 2016 Bourgogne Laforet,580,2,2,1160,已確認
2,徐筱熙,506944678,20190812,201908120001,Derek Chen,4496,La Chablisienne 2004 Chablis 1er Cru Fourchaume,790,2,2,1580,已確認
3,徐筱熙,506944678,20190812,201908120001,總計款數,6,自取付款金額,4100,運費,0,宅配付款金額,4100
4,佩玄,517368306,20190812,201908120002,Youchun Chen,4507,Joseph Drouhin 2016 Bourgogne Laforet,580,1,1,580,已確認
5,佩玄,517368306,20190812,201908120002,Youchun Chen,4485,Henri Giraud NV Esprit Nature,1250,1,1,1250,已確認
6,佩玄,517368306,20190812,201908120002,Youchun Chen,4496,La Chablisienne 2004 Chablis 1er Cru Fourchaume,790,1,1,790,已確認
7,佩玄,517368306,20190812,201908120002,總計款數,3,自取付款金額,2620,運費,225,宅配付款金額,2845
8,陳曉丰,526487238,20190812,201908120003,Leo Huang,4417,Ponsot 2013 Morey Saint Denis 1er Cru Les Luis...,2990,1,1,2990,已確認
9,陳曉丰,526487238,20190812,201908120003,總計款數,1,自取付款金額,2990,運費,160,宅配付款金額,3150
